# Preliminaries

This is to ensure that torch recognizes macOS software. 

In [2]:
import torch 
print("PyTorch Ver:", torch.__version__) 
print("Apple Silicon available:", torch.backends.mps.is_available())

PyTorch Ver: 2.6.0
Apple Silicon available: True
